In [1]:
import pandas as pd

In [3]:
df = pd.read_excel("Dataset_Datathon_Fraud_Detection_28122022.xlsx")

In [5]:
df.describe

<bound method NDFrame.describe of            customer_id     event_dttm_Deli    event_type_nm  \
0        customer_id_0 2022-10-01 10:51:36  CHANGE_PASSWORD   
1        customer_id_0 2022-10-01 11:15:21   SESSION_SIGNIN   
2        customer_id_0 2022-10-01 11:31:55   SESSION_SIGNIN   
3        customer_id_0 2022-10-01 11:45:42   SESSION_SIGNIN   
4        customer_id_0 2022-10-03 10:28:24          DEPOSIT   
...                ...                 ...              ...   
10964  customer_id_199 2022-10-29 01:22:11   SESSION_SIGNIN   
10965  customer_id_199 2022-10-29 12:12:53   SESSION_SIGNIN   
10966  customer_id_199 2022-10-29 12:13:27          PAYMENT   
10967  customer_id_199 2022-10-29 15:26:58   SESSION_SIGNIN   
10968  customer_id_199 2022-10-31 21:55:16          PAYMENT   

      client_defined_event_type_nm               event_desc  \
0                            RENEW  Login password recovery   
1                   SESSION_SIGNIN                    Login   
2                   

In [6]:
df.columns

Index(['customer_id', 'event_dttm_Deli', 'event_type_nm',
       'client_defined_event_type_nm', 'event_desc',
       'operaton_amt_100 (rupees)', 'active phone call',
       'remote desktop protokol', 'emulator', 'f', 'device_id', 'os',
       'device_model', 'screensize', 'agentconnectiontype', 'screen_hash',
       'device for user', 'reciever', 'password change', 'fast change of ip',
       'ip not equal to previous'],
      dtype='object')

In [12]:
df["ip not equal to previous"]

0        IP differes
1        IP differes
2        IP differes
3        IP differes
4                NaN
            ...     
10964    IP differes
10965    IP differes
10966    IP differes
10967    IP differes
10968            NaN
Name: ip not equal to previous, Length: 10969, dtype: object

In [77]:
df.sort_values(['customer_id', 'event_dttm_Deli'], inplace=True)
df['new_session'] = (df['event_type_nm'] == 'SESSION_SIGNIN').astype(int)
df['session_num'] = df.groupby('customer_id')['new_session'].cumsum()
df.drop(['new_session'], axis=1, inplace=True)

In [161]:
from scipy import stats

In [ ]:
import numpy as np

In [155]:
df.fillna('a').groupby(['customer_id','session_num']).agg(df['device_model'].max())

TypeError: '>=' not supported between instances of 'float' and 'str'

In [209]:
session_df = df.fillna('').groupby(['customer_id','session_num']).agg(
    timestamp_min = ('event_dttm_Deli',min),timestamp_max = ('event_dttm_Deli',max),
    event_desc = ('event_desc',lambda x: ','.join(x)),
    amount_sum = ('operaton_amt_100 (rupees)',lambda x: sum(x)),
    curr_device = ('device_model',lambda x: max(x)),#value_counts().index[0] if x.value_counts().index else '')
    curr_location = ('f','max')
#     payment_cnt = ('event_desc',lambda x: sum(('Purchase' in x) | ('Payment' in x)))
)

In [210]:
session_df

timestamp_min       timestamp_max  \
customer_id    session_num                                           
customer_id_0  0           2022-10-01 10:51:36 2022-10-01 10:51:36   
               1           2022-10-01 11:15:21 2022-10-01 11:15:21   
               2           2022-10-01 11:31:55 2022-10-01 11:31:55   
               3           2022-10-01 11:45:42 2022-10-03 10:28:24   
               4           2022-10-03 10:29:37 2022-10-03 10:29:37   
...                                        ...                 ...   
customer_id_99 20          2022-11-18 18:35:22 2022-11-18 20:14:57   
               21          2022-11-18 20:20:19 2022-11-19 18:32:44   
               22          2022-11-20 00:08:33 2022-11-20 00:08:33   
               23          2022-11-20 15:36:10 2022-11-20 15:47:50   
               24          2022-11-20 20:11:08 2022-11-20 20:11:08   

                                                                   event_desc  \
customer_id    session_num                                                      
customer_id_0  0                                      Login password recovery   
               1                                                        Login   
               2                                                        Login   
               3                                          Login,Payment order   
               4                                                        Login   
...                                                                       ...   
customer_id_99 20                            Login,Purchase,Purchase,Purchase   
               21           Login,Payment between Telebank clients (Person...   
               22                                                       Login   
               23                                              Login,Purchase   
               24                                                       Login   

                            amount_sum curr_device  curr_location  
customer_id    session_num                                         
customer_id_0  0                  0.00              St Petersburg  
               1                  0.00              St Petersburg  
               2                  0.00              St Petersburg  
               3             116820.00      iPhone  St Petersburg  
               4                  0.00              St Petersburg  
...                                ...         ...            ...  
customer_id_99 20             17223.28      iPhone         Moscow  
               21             78962.06      iPhone         Moscow  
               22                 0.00      iPhone         Moscow  
               23              1294.46      iPhone         Moscow  
               24                 0.00      iPhone         Moscow  

[2746 rows x 6 columns]

In [211]:
session_df['duration'] = (session_df['timestamp_max'] - session_df['timestamp_min']).dt.total_seconds()

In [180]:
session_df

timestamp_min       timestamp_max  \
customer_id    session_num                                           
customer_id_0  0           2022-10-01 10:51:36 2022-10-01 10:51:36   
               1           2022-10-01 11:15:21 2022-10-01 11:15:21   
               2           2022-10-01 11:31:55 2022-10-01 11:31:55   
               3           2022-10-01 11:45:42 2022-10-03 10:28:24   
               4           2022-10-03 10:29:37 2022-10-03 10:29:37   
...                                        ...                 ...   
customer_id_99 20          2022-11-18 18:35:22 2022-11-18 20:14:57   
               21          2022-11-18 20:20:19 2022-11-19 18:32:44   
               22          2022-11-20 00:08:33 2022-11-20 00:08:33   
               23          2022-11-20 15:36:10 2022-11-20 15:47:50   
               24          2022-11-20 20:11:08 2022-11-20 20:11:08   

                                                                   event_desc  \
customer_id    session_num                                                      
customer_id_0  0                                      Login password recovery   
               1                                                        Login   
               2                                                        Login   
               3                                          Login,Payment order   
               4                                                        Login   
...                                                                       ...   
customer_id_99 20                            Login,Purchase,Purchase,Purchase   
               21           Login,Payment between Telebank clients (Person...   
               22                                                       Login   
               23                                              Login,Purchase   
               24                                                       Login   

                            amount_sum curr_device  duration  
customer_id    session_num                                    
customer_id_0  0                  0.00                   0.0  
               1                  0.00                   0.0  
               2                  0.00                   0.0  
               3             116820.00      iPhone  168162.0  
               4                  0.00                   0.0  
...                                ...         ...       ...  
customer_id_99 20             17223.28      iPhone    5975.0  
               21             78962.06      iPhone   79945.0  
               22                 0.00      iPhone       0.0  
               23              1294.46      iPhone     700.0  
               24                 0.00      iPhone       0.0  

[2746 rows x 6 columns]

In [212]:
session_df['event_desc'] = session_df['event_desc'].str.lower()

In [213]:
session_df['event_desc_flag'] = session_df['event_desc'].str.contains('password') & (session_df['event_desc'].str.contains('payment') | session_df['event_desc'].str.contains('purchase'))

In [198]:
session_df

timestamp_min       timestamp_max  \
customer_id    session_num                                           
customer_id_0  0           2022-10-01 10:51:36 2022-10-01 10:51:36   
               1           2022-10-01 11:15:21 2022-10-01 11:15:21   
               2           2022-10-01 11:31:55 2022-10-01 11:31:55   
               3           2022-10-01 11:45:42 2022-10-03 10:28:24   
               4           2022-10-03 10:29:37 2022-10-03 10:29:37   
...                                        ...                 ...   
customer_id_99 20          2022-11-18 18:35:22 2022-11-18 20:14:57   
               21          2022-11-18 20:20:19 2022-11-19 18:32:44   
               22          2022-11-20 00:08:33 2022-11-20 00:08:33   
               23          2022-11-20 15:36:10 2022-11-20 15:47:50   
               24          2022-11-20 20:11:08 2022-11-20 20:11:08   

                                                                   event_desc  \
customer_id    session_num                                                      
customer_id_0  0                                      login password recovery   
               1                                                        login   
               2                                                        login   
               3                                          login,payment order   
               4                                                        login   
...                                                                       ...   
customer_id_99 20                            login,purchase,purchase,purchase   
               21           login,payment between telebank clients (person...   
               22                                                       login   
               23                                              login,purchase   
               24                                                       login   

                            amount_sum curr_device  duration  event_desc_flag  
customer_id    session_num                                                     
customer_id_0  0                  0.00                   0.0            False  
               1                  0.00                   0.0            False  
               2                  0.00                   0.0            False  
               3             116820.00      iPhone  168162.0            False  
               4                  0.00                   0.0            False  
...                                ...         ...       ...              ...  
customer_id_99 20             17223.28      iPhone    5975.0            False  
               21             78962.06      iPhone   79945.0            False  
               22                 0.00      iPhone       0.0            False  
               23              1294.46      iPhone     700.0            False  
               24                 0.00      iPhone       0.0            False  

[2746 rows x 7 columns]

In [214]:
from statistics import mean,mode,stdev

In [215]:
customer_df = df.fillna('').groupby(['customer_id']).agg(
    amount_stdev = ('operaton_amt_100 (rupees)',stdev),amount_mean = ('operaton_amt_100 (rupees)','mean'),device = ('device_model',max),operating_sys = ('os','max'),location = ('f','max') 
)

In [201]:
customer_df

,amount_stdev,amount_mean,device,operating_sys,location
customer_id,,,,,
customer_id_0,172225.843084,110242.297297,iPhone,iOS,St Petersburg
customer_id_1,194162.236401,131882.352941,,,Puna
customer_id_10,537854.119863,202693.037826,Xiaomi 2112123AG,Android,Novosibirsk
customer_id_100,12095.176122,6516.657273,iPhone,iOS,Surat
customer_id_101,239.351856,135.615714,HUAWEI MOA-LX9N,Android,Akhmadabad
...,...,...,...,...,...
customer_id_95,333178.743445,227458.882353,Xiaomi M2007J20CG,Android,Vladivostok
customer_id_96,38980.005319,10605.243789,HUAWEI HRY-LX1T,Android,Agra
customer_id_97,429196.776516,180304.726154,iPhone,iOS,Moscow


In [216]:
df_merged = pd.merge(session_df, customer_df, left_on='customer_id', right_on='customer_id', how='left')#.drop('id1', axis=1)

In [217]:
df_merged

,timestamp_min,timestamp_max,event_desc,amount_sum,curr_device,curr_location,duration,event_desc_flag,amount_stdev,amount_mean,device,operating_sys,location
customer_id,,,,,,,,,,,,,
customer_id_0,2022-10-01 10:51:36,2022-10-01 10:51:36,login password recovery,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg
customer_id_0,2022-10-01 11:15:21,2022-10-01 11:15:21,login,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg
customer_id_0,2022-10-01 11:31:55,2022-10-01 11:31:55,login,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg
customer_id_0,2022-10-01 11:45:42,2022-10-03 10:28:24,"login,payment order",116820.00,iPhone,St Petersburg,168162.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg
customer_id_0,2022-10-03 10:29:37,2022-10-03 10:29:37,login,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
customer_id_99,2022-11-18 18:35:22,2022-11-18 20:14:57,"login,purchase,purchase,purchase",17223.28,iPhone,Moscow,5975.0,False,37321.371793,15444.667761,iPhone,iOS,Moscow
customer_id_99,2022-11-18 20:20:19,2022-11-19 18:32:44,"login,payment between telebank clients (person...",78962.06,iPhone,Moscow,79945.0,False,37321.371793,15444.667761,iPhone,iOS,Moscow
customer_id_99,2022-11-20 00:08:33,2022-11-20 00:08:33,login,0.00,iPhone,Moscow,0.0,False,37321.371793,15444.667761,iPhone,iOS,Moscow


In [218]:
df_merged['amount_flag'] = df_merged['amount_sum'] > df_merged['amount_stdev'] 

In [219]:
df_merged['device_flag'] = df_merged['curr_device'] != df_merged['device'] 

In [220]:
df_merged['location_flag'] = df_merged['curr_location'] != df_merged['location'] 

In [221]:
df_merged

,timestamp_min,timestamp_max,event_desc,amount_sum,curr_device,curr_location,duration,event_desc_flag,amount_stdev,amount_mean,device,operating_sys,location,amount_flag,device_flag,location_flag
customer_id,,,,,,,,,,,,,,,,
customer_id_0,2022-10-01 10:51:36,2022-10-01 10:51:36,login password recovery,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg,False,True,False
customer_id_0,2022-10-01 11:15:21,2022-10-01 11:15:21,login,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg,False,True,False
customer_id_0,2022-10-01 11:31:55,2022-10-01 11:31:55,login,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg,False,True,False
customer_id_0,2022-10-01 11:45:42,2022-10-03 10:28:24,"login,payment order",116820.00,iPhone,St Petersburg,168162.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg,False,False,False
customer_id_0,2022-10-03 10:29:37,2022-10-03 10:29:37,login,0.00,,St Petersburg,0.0,False,172225.843084,110242.297297,iPhone,iOS,St Petersburg,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
customer_id_99,2022-11-18 18:35:22,2022-11-18 20:14:57,"login,purchase,purchase,purchase",17223.28,iPhone,Moscow,5975.0,False,37321.371793,15444.667761,iPhone,iOS,Moscow,False,False,False
customer_id_99,2022-11-18 20:20:19,2022-11-19 18:32:44,"login,payment between telebank clients (person...",78962.06,iPhone,Moscow,79945.0,False,37321.371793,15444.667761,iPhone,iOS,Moscow,True,False,False
customer_id_99,2022-11-20 00:08:33,2022-11-20 00:08:33,login,0.00,iPhone,Moscow,0.0,False,37321.371793,15444.667761,iPhone,iOS,Moscow,False,False,False


In [222]:
df_merged.to_csv('final_merged.csv')